# Strava Activities Downloader

Download all of your Strava Activites and Export to CSV

SEE: [strava_data_analysis.ipynb](https://github.com/markwk/qs_ledger/blob/master/strava/strava_data_analysis.ipynb) for Strava Data Analysis and Data Visualization of Running, Cycling and Other Activities

-----

## References: 

* https://developers.strava.com/docs/
* Alternative Library: https://github.com/hozn/stravalib/

------

## Dependencies and Libraries

In [1]:
import requests
import time
import datetime
from datetime import date
import collections
import json

In [2]:
import numpy as np, string, re, pytz
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

------

## Authorization / Credentials

In [3]:
import json

with open("credentials.json", "r") as file:
    credentials = json.load(file)
    strava_cr = credentials['strava']
    ACCESS_TOKEN = strava_cr['ACCESS_TOKEN']

-----

# Athlete Info

SEE: https://developers.strava.com/docs/reference/#api-Athletes

In [4]:
def get_strava_athlete():
    url = 'https://www.strava.com/api/v3/athlete/'
    header = {'Authorization': 'Bearer '+ACCESS_TOKEN}
    param = {'per_page':200, 'page':1}
    dataset = requests.get(url, headers=header, params=param).json()
    return dataset

In [5]:
athlete = get_strava_athlete()

In [6]:
firstname = athlete.get('firstname')
athlete_id = athlete.get('id')
print(f"Hi, {firstname} (ID: {athlete_id})")

Hi, Mark (ID: 13740897)


----

# Athlete Stats

SEE: https://developers.strava.com/docs/reference/#api-Athletes-getStats

In [7]:
def get_strava_athlete_stats(athlete_id):
    url = 'https://www.strava.com/api/v3/athletes/' + str(athlete_id) + '/stats/'
    header = {'Authorization': 'Bearer '+ACCESS_TOKEN}
    param = {'per_page':200, 'page':1}
    dataset = requests.get(url, headers=header, params=param).json()
    return dataset

In [8]:
athlete_stats_dict = get_strava_athlete_stats(athlete_id)

In [9]:
# print(athlete_stats_dict)

----

# Get Basic Activities from Strava

Get Basic Activites Summaries from Strava and Export to CSV. 

SEE: https://developers.strava.com/docs/reference/#api-Activities

In [10]:
def convert_timestamp(i):
    return datetime.strptime(i, "%Y-%m-%dT%H:%M:%SZ")

def event_timestamp(i):
    return convert_timestamp(i['start_date_local'])

In [11]:
def get_strava_activites():
    url = 'https://www.strava.com/api/v3/athlete/activities'
    header = {'Authorization': 'Bearer '+ACCESS_TOKEN}
    param = {'per_page':200, 'page':1}
    dataset = requests.get(url, headers=header, params=param).json()
    count = len(dataset)
    if count == 200: #if 200 results come back
        loop_count = 1 #we've already done one loop
        while count == 200: #while it keeps returning 200 results
            loop_count = loop_count + 1 #increase loop_count or page number
            param = {'per_page':200, 'page':loop_count} #reset params
            sub_dataset = requests.get(url, headers=header, params=param).json() #pull new data with sub_dataset name
            dataset = dataset + sub_dataset #combine (Json files, not dictionaries thank jesus)
            count = len(sub_dataset) #count results to see if we need to loop again
    # return {event_timestamp(i): clean_event(i) for i in dataset if wanted_event(i)} #return as normal
    return {event_timestamp(i): i for i in dataset}

In [12]:
# Get Activties Data from Strava
strava_dict = get_strava_activites()
print(f"Extracted {len(strava_dict)} activities from Strava.")

Extracted 833 activities from Strava.


In [13]:
# Create DF and Export to CSV
strava_activities = pd.DataFrame.from_dict(strava_dict, orient='index')

In [14]:
strava_activities.to_csv('data/strava-activities-raw.csv')

In [17]:
strava_activities.tail()

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,...,display_hide_heartrate_option,elev_high,elev_low,pr_count,total_photo_count,has_kudoed,average_watts,kilojoules,device_watts,average_cadence
2018-12-28 14:51:33,2,"{'id': 13740897, 'resource_state': 1}",Afternoon Run,4245.7,1807,1807,0.0,Run,0.0,2039524530,...,False,NaN,NaN,0,0,False,NaN,NaN,NaN,NaN
2018-12-28 15:31:32,2,"{'id': 13740897, 'resource_state': 1}","Fitbod - Quadriceps, Hamstrings, Shoulders, Ab...",0.0,1222,1222,0.0,WeightTraining,NaN,2039545103,...,False,NaN,NaN,0,1,False,NaN,NaN,NaN,NaN
2018-12-29 17:21:41,2,"{'id': 13740897, 'resource_state': 1}",Afternoon Run,5018.0,1886,1973,49.9,Run,0.0,2041694063,...,False,36.9,12.8,0,0,False,NaN,NaN,NaN,NaN
2018-12-31 17:37:04,2,"{'id': 13740897, 'resource_state': 1}",Afternoon Run,4014.0,1413,1424,26.5,Run,0.0,2046222812,...,False,20.6,2.7,0,0,False,NaN,NaN,NaN,NaN
2018-12-31 18:26:27,2,"{'id': 13740897, 'resource_state': 1}",Evening Run,2282.4,788,791,19.8,Run,0.0,2046222800,...,False,13.1,3.8,0,0,False,NaN,NaN,NaN,NaN


-----

# Get Detailed Activity from Strava

SEE: https://developers.strava.com/docs/reference/#api-Activities-getActivityById

In [18]:
test_activity_id = strava_activities.tail(1).iloc[0]['id']

In [19]:
def get_strava_activity(id):
    url = 'https://www.strava.com/api/v3/activities/' + str(id)
    header = {'Authorization': 'Bearer '+ACCESS_TOKEN}
    param = {'per_page':200, 'page':1}
    dataset = requests.get(url, headers=header, params=param).json()
    return dataset

In [22]:
activity = get_strava_activity(1970837949)
# activity

In [24]:
# activity